In [1]:
from fastembed import SparseTextEmbedding
import os

from RAG.RAG_query import get_topics_list
from config.loadConfig import load_project_config
from DB.qDrant import check_qdrant_status, QdrantConfig
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from RAG.RAG_Retrieve import custom_embed_function
# Encode the query text using the same model

# Connect to Qdrant
config = load_project_config(os.path.join(os.getcwd(), "config/config.yaml"))
qdrantConfig = QdrantConfig(
    host=config["db"]["qdrant"]["connection"]["host"],
    port=config["db"]["qdrant"]["connection"]["port"],
    timeout=config["db"]["qdrant"]["connection"]["timeout"]
)
status, client = check_qdrant_status(qdrantConfig)
embedding_model = SentenceTransformer(config['embedding_model'])
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm25")
small_collection_name = config['db']['collections']['collection_small']['name']
large_collection_name = config['db']['collections']['collection_large']['name']

Connected. Collections: ['wiki_large_chunks', 'wiki_small_chunks']


In [17]:
get_topics_list(client)

0000d22f-a0a6-446c-b753-2cb8b252bbcb Andre Agassi
00164fc4-39be-4ef0-8d91-39690d7d7c89 Alabama
0024beeb-8091-4ef4-9e54-8be4518b170c Animal Farm
0043c428-fb58-4b7b-9a0f-eed75dccf897 Alabama
007f0cdd-9081-4987-80bc-b46b47c6e033 Abraham Lincoln
007faa8c-f372-4ef7-8d4f-43f6b6cdc744 Afroasiatic languages
00d8a63b-013e-4dc5-bbfc-138a2599835c Aristotle
00ebad82-b699-407c-afb9-6b9ae7fa5919 Alabama
00fa2926-5539-4954-b04c-eca0e6e4ea47 Animation
01145695-2e8d-4f5d-bd0a-c6e7965c8490 Andre Agassi
0000d22f-a0a6-446c-b753-2cb8b252bbcb Andre Agassi
00164fc4-39be-4ef0-8d91-39690d7d7c89 Alabama
0024beeb-8091-4ef4-9e54-8be4518b170c Animal Farm
0043c428-fb58-4b7b-9a0f-eed75dccf897 Alabama
007f0cdd-9081-4987-80bc-b46b47c6e033 Abraham Lincoln
007faa8c-f372-4ef7-8d4f-43f6b6cdc744 Afroasiatic languages
00d8a63b-013e-4dc5-bbfc-138a2599835c Aristotle
00ebad82-b699-407c-afb9-6b9ae7fa5919 Alabama
00fa2926-5539-4954-b04c-eca0e6e4ea47 Animation
01145695-2e8d-4f5d-bd0a-c6e7965c8490 Andre Agassi


In [8]:
from RAG.RAG_query import sparse_query_search

query_text = "Where is alabama?"
results = sparse_query_search(client, query_text)


--- : Querying Small Chunks for: 'Where is alabama?' ---
External links        Alabama State Guide, from the Library of Congress  Your Not So Ordinary Alabama Tourist Guide   All About Alabama , at the Alabama Department of Archives and History  Code of Alabama 1975  USGS real-time, geographic, and other scientific resources of Alabama   Alabama QuickFacts from the U.S. Census Bureau  Alabama State Fact Sheet
The executive branch is responsible for the execution and oversight of laws. It is headed by the governor of Alabama. Other members of the executive branch include the cabinet, the lieutenant governor of Alabama, the Attorney General of Alabama, the Alabama Secretary of State, the Alabama State Treasurer, and the State Auditor of Alabama. The current governor is Republican Kay Ivey.
Water ports of Alabama, listed from north to south:  See also    Index of Alabama-related articles  Outline of Alabama — organized list of topics about Alabama  USS Alabama, 7 ships  Notes  Subnotes  

In [10]:
result = client.retrieve(
    collection_name="wiki_large_chunks",
    ids=["035951e1-74e1-450e-a6a8-5bf9440fbad1"],
    with_payload=True
)
print(result)

[Record(id='035951e1-74e1-450e-a6a8-5bf9440fbad1', payload={'text': 'Alabama () is a state in the Southeastern region of the United States, bordered by Tennessee to the north; Georgia to the east; Florida and the Gulf of Mexico to the south; and Mississippi to the west. Alabama is the 30th largest by area and the 24th-most populous of the U.S. states.\n\nAlabama is nicknamed the Yellowhammer State, after the state bird. Alabama is also known as the "Heart of Dixie" and the "Cotton State". The state tree is the longleaf pine, and the state flower is the camellia. Alabama\'s capital is Montgomery, and its largest city by population and area is Huntsville. Its oldest city is Mobile, founded by French colonists (Alabama Creoles) in 1702 as the capital of French Louisiana. Greater Birmingham is Alabama\'s largest metropolitan area and its economic center.', 'title': 'Alabama'}, vector=None, shard_key=None, order_value=None)]
